In [3]:
import pandas as pd 

In [4]:
from sqlparsemodel.Sqlstructure import Sqlstructure
from sqlparsemodel.Sastructure import Sastructure
from sqlparsemodel.Mermaidplot import Sqltoflowchart
from sqlparsemodel.Querystate import Querystate
import re 
Querystate = Querystate()

In [5]:
query = '''


PROC SQL;
   CREATE TABLE MEOW.PAST_CLAIM_APLY_NO AS 
   SELECT DISTINCT t1.CONTRACT_NO,
          t1.INSD_ID_SAS,
          t2.APLY_NO
      FROM Meow.POL_ALL_X_01
           INNER JOIN QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS)
      WHERE t1.CREATE_DATE > t2.OCR_DATE ;
QUIT;



PROC SQL;
   CREATE TABLE WORK.PAST_CLAIM_ICDS AS 
   SELECT DISTINCT 
          t1.CONTRACT_NO,
          t1.INSD_ID_SAS,
          t2.APLY_NO, 
          t2.DIAG_SER_NO, 
          t2.CFM_ICD_CODE1, 
          t2.CFM_ICD_CODE2, 
          t2.CFM_ICD_CODE3, 
          t2.CFM_ICD_CODE4, 
          t2.CFM_ICD_CODE5
      FROM MEOW.PAST_CLAIM_APLY_NO t1
	       INNER JOIN (SELECT * FROM QUERY.DTAAA020_AE) t2 ON (t1.APLY_NO = t2.APLY_NO);
QUIT;'''

query = '''


PROC SQL;
   CREATE TABLE MEOW.PAST_CLAIM_APLY_NO AS 
   SELECT DISTINCT t1.CONTRACT_NO,
          t1.INSD_ID_SAS,
          t2.APLY_NO
      FROM Meow.POL_ALL_X_01
           INNER JOIN QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS)
      WHERE t1.CREATE_DATE > t2.OCR_DATE ;
QUIT;



PROC SQL;
   CREATE TABLE WORK.PAST_CLAIM_ICDS AS 
   SELECT DISTINCT 
          t1.CONTRACT_NO,
          t1.INSD_ID_SAS,
          t2.APLY_NO, 
          t2.DIAG_SER_NO, 
          t2.CFM_ICD_CODE1, 
          t2.CFM_ICD_CODE2, 
          t2.CFM_ICD_CODE3, 
          t2.CFM_ICD_CODE4, 
          t2.CFM_ICD_CODE5
      FROM MEOW.PAST_CLAIM_APLY_NO t1
	       INNER JOIN QUERY.DTAAA020_AE t2 ON (t1.APLY_NO = t2.APLY_NO);
QUIT;'''

query3 = '''
PROC SQL;
   CREATE TABLE Meow.product_mkt_type_cd AS 
   SELECT t1.INSURANCE_PRODUCT_CD, 
          t1.PRODUCT_MKT_M_TYPE_CD
      FROM CR20.LIFE_INSURANCE_PRODUCT_V t1
	  WHERE  t1.X_SYS_TYPE = '1'
      ORDER BY t1.INSURANCE_PRODUCT_CD;
QUIT;
PROC SQL;
   CREATE TABLE Meow.Main_policy AS 
   SELECT t1.INSURANCE_PRODUCT_CD, 
                 t1.PRODUCT_MKT_M_TYPE_CD
      FROM CR20.LIFE_INSURANCE_PRODUCT_V t1
	  WHERE  t1.X_SYS_TYPE = '1' AND t1.BASE_COVERAGE_INDICATOR_CD='1'
      ORDER BY t1.INSURANCE_PRODUCT_CD;
QUIT;
PROC SQL;
   CREATE TABLE Meow.ADD_policy AS 
   SELECT t1.INSURANCE_PRODUCT_CD, 
          t1.PRODUCT_MKT_M_TYPE_CD
      FROM CR20.LIFE_INSURANCE_PRODUCT_V t1
	  WHERE  t1.X_SYS_TYPE = '1' AND t1.BASE_COVERAGE_INDICATOR_CD='2'
      ORDER BY t1.INSURANCE_PRODUCT_CD;
QUIT;
PROC SQL;
   CREATE TABLE Meow.ALL_policy AS 
   SELECT t1.INSURANCE_PRODUCT_CD, 
          t1.PRODUCT_MKT_M_TYPE_CD
      FROM CR20.LIFE_INSURANCE_PRODUCT_V t1
	  WHERE  t1.X_SYS_TYPE = '1' 
      ORDER BY t1.INSURANCE_PRODUCT_CD;
QUIT;

'''

query4 = '''

PROC SQL;
   CREATE TABLE Meow.HOSPITAL_5Y_01 AS 
   SELECT DISTINCT t1.CONTRACT_NO, 
         t1.CREATE_DATE,
          t1.INSD_ID_SAS,
          t2.APLY_NO,
		  t2.POLICY_NO AS POL_2, 
		  t2.PROD_ID,
	      t2.OCR_DATE,
		  t2.E_STR_DATE,
		  t2.E_END_DATE,
		  t2.CLAM_AMT_CODE,
		  t2.CLAM_AMT_NAME,
		  t2.CLAM_CAT,
		  t2.OCR_RESN, 
          t2.BONE_CODE, 
          t2.BONE_NAME, 
		  t2.OP_CODE,
          t2.OP_NAME,
		  t2.PAY_DAY,
		  t2.PAY_AMT,
		  t2.PAY_STS
      FROM Meow.POL_ALL_X_01 t1
           LEFT JOIN QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS)
      WHERE t1.CREATE_DATE > t2.ISSUE_DATE ;
QUIT;

PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_02 AS 
   SELECT DISTINCT  t1.CONTRACT_NO, 
           t1.CREATE_DATE, 
          t1.INSD_ID_SAS,
          t1.APLY_NO,
		  t1.POL_2 , 
		  t1.PROD_ID,
	      t1.OCR_DATE,
		  t1.E_STR_DATE,
		  t1.E_END_DATE,
		  t1.OCR_RESN, 
          t1.BONE_CODE, 
          t1.BONE_NAME, 
		   t1.CLAM_AMT_CODE,
		  t1.CLAM_AMT_NAME,
		  t1.OP_CODE,
          t1.OP_NAME,
		  t1.PAY_DAY
      FROM Meow.HOSPITAL_5Y_01 t1
      WHERE t1.PAY_AMT > 0 AND t1.PAY_STS NOT = '5'  AND CLAM_AMT_CODE in( 'BEE1' , 'BEF1') AND  t1.CLAM_CAT in ('E' ,'F') AND t1.BONE_CODE ='' ;
QUIT;


PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_03 AS 
   SELECT DISTINCT  t1.CONTRACT_NO,
          t1.INSD_ID_SAS,
           t1.CREATE_DATE, 
          t1.POL_2, 
          t1.OCR_DATE, 
		  t1.E_STR_DATE,
		  t1.E_END_DATE,
		    /* E_STR_DATE_NEW */
		  (CASE
         WHEN t1.E_STR_DATE NOT IS MISSING THEN t1.E_STR_DATE
          ELSE t1.OCR_DATE END)  FORMAT=date9. AS E_STR_DATE_NEW,
  /* E_END_DATE_NEW */
		(CASE 
		WHEN t1.E_STR_DATE NOT IS MISSING THEN t1.E_STR_DATE
		ELSE t1.OCR_DATE
		END)+t1.PAY_DAY-1  FORMAT=date9. AS E_END_DATE_NEW,
		   t1.CLAM_AMT_CODE, 
          t1.CLAM_AMT_NAME,
		  t1.OCR_RESN, 
          t1.BONE_CODE, 
          t1.BONE_NAME,
          t1.PAY_DAY
      FROM WORK.HOSPITAL_5Y_02 t1;
QUIT;

PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_04_year_0to1 AS 
   SELECT DISTINCT  t1.CONTRACT_NO,
         t1.INSD_ID_SAS, 
		  t1.CREATE_DATE,
          t1.POL_2, 
		  t1.OCR_RESN, 
	      t1.E_STR_DATE_NEW,
         t1.E_END_DATE_NEW,
          t1.PAY_DAY
      FROM WORK.HOSPITAL_5Y_03 t1
	  WHERE t1.E_END_DATE_NEW >=   t1.CREATE_DATE-365*1 AND t1.E_STR_DATE_NEW <  t1.CREATE_DATE ;
QUIT;

PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_04_year_1to2 AS 
   SELECT DISTINCT t1.CONTRACT_NO,
         t1.INSD_ID_SAS, 
		  t1.CREATE_DATE,
          t1.POL_2, 
		  t1.OCR_RESN, 
	      t1.E_STR_DATE_NEW,
         t1.E_END_DATE_NEW,
          t1.PAY_DAY
      FROM WORK.HOSPITAL_5Y_03 t1
	  WHERE t1.E_END_DATE_NEW >=  t1.CREATE_DATE-365*2 AND t1.E_STR_DATE_NEW <  t1.CREATE_DATE-365*1 ;
QUIT;

'''

query5 = '''

PROC SQL;
   CREATE TABLE WORK.HOSPITAL_5Y_04_year_1to2 AS 
   SELECT DISTINCT t1.CONTRACT_NO,
         t1.INSD_ID_SAS, 
		  t1.CREATE_DATE,
          t1.POL_2, 
		  t1.OCR_RESN, 
	      t1.E_STR_DATE_NEW,
         t1.E_END_DATE_NEW,
          t1.PAY_DAY
      FROM WORK.HOSPITAL_5Y_03 t1
	  WHERE t1.E_END_DATE_NEW >=  t1.CREATE_DATE-365*2 AND t1.E_STR_DATE_NEW <  t1.CREATE_DATE-365*1 ;
QUIT;


'''

query6 = '''
SELECT * From 
(
  SELECT  *, (Dense_Rank() OVER (ORDER BY Column ASC)) AS Rank, b
  FROM Table1 
) AS Ranking 
WHERE Rank % 10 = 0;

'''

query = re.sub("\\/.*?\\/", "",query6)

# query to Structure List to SAS node relation 
sas_query = Sastructure(query)

# Node relation Setting and Plot the mermaid structure
sfc = Sqltoflowchart(sas_query.from_to_property,
 sas_query.from_to_query,
 sas_query.node_property )

# 1. Ignore Some Relation words 
# 2. Draw some node to colors 
sfc.mermaid_plot(relation_ignore = [])
sfc.mermaid_drawnode(keyword = "QUERY", token_tag = "token", color = "#28FF28")
sfc.mermaid_drawnode(keyword = "CR20", token_tag = "token", color = "#FFAA33")
sfc.mc.mermaid_txt

SELECT * From 
(
  SELECT  *, (Dense_Rank() OVER (ORDER BY Column ASC)) AS Rank, b
  FROM Table1 
) AS Ranking 
WHERE Rank % 10 = 0;
{}
|- 0 Newline ' '
|- 1 DML 'SELECT'
|- 2 Whitespace ' '
|- 3 Wildcard '*'
|- 4 Whitespace ' '
|- 5 Keyword 'From'
|- 6 Whitespace ' '
|- 7 Newline ' '
|- 8 Identifier '( SE...'
|  |- 0 Parenthesis '( SE...'
|  |  |- 0 Punctuation '('
|  |  |- 1 Newline ' '
|  |  |- 2 Whitespace ' '
|  |  |- 3 Whitespace ' '
|  |  |- 4 DML 'SELECT'
|  |  |- 5 Whitespace ' '
|  |  |- 6 Whitespace ' '
|  |  |- 7 IdentifierList '*, (De...'
|  |  |  |- 0 Wildcard '*'
|  |  |  |- 1 Punctuation ','
|  |  |  |- 2 Whitespace ' '
|  |  |  |- 3 Identifier '(Dense...'
|  |  |  |  |- 0 Parenthesis '(Dense...'
|  |  |  |  |  |- 0 Punctuation '('
|  |  |  |  |  |- 1 Function 'Dense_...'
|  |  |  |  |  |  |- 0 Identifier 'Dense_...'
|  |  |  |  |  |  |  `- 0 Name 'Dense_...'
|  |  |  |  |  |  `- 1 Parenthesis '()'
|  |  |  |  |  |     |- 0 Punctuation '('
|  |  |  |  |  |     `- 1 Punc

'graph TD;subgraph NOCREATETB.query1;2(<center><b>FROM</b><br>SUBQ.MAIN_FROM1, Ranking</center>)-->1{<center><b>FROM</b></center>};1{<center><b>FROM</b></center>}-->|<center>SUBQ.MAIN_FROM1 AS Ranking</center>|3{<center><b>WHERE</b></center>};3{<center><b>WHERE</b></center>}-->|<center>WHERE Rank % 10 = 0</center>|4{<center><b>SELECT</b></center>};4{<center><b>SELECT</b></center>}-->|<center>*</center>|5(<center><b>TABLE</b><br>NOCREATETB.query1, </center>);end; subgraph SUBQ.MAIN_FROM1;7(<center><b>FROM</b><br>.Table1, </center>)-->6{<center><b>FROM</b></center>};6{<center><b>FROM</b></center>}-->|<center>Table1</center>|8{<center><b>SELECT</b></center>};8{<center><b>SELECT</b></center>}-->|<center>*, <br>  Dense_Rank   OVER  ORDER BY Column ASC   AS Rank, <br> b</center>|9(<center><b>TABLE</b><br>SUBQ.MAIN_FROM1, </center>);end; 9-.->2;'

In [6]:
pd.DataFrame(sas_query.node_property)

,0,1,2,3,4,5,6,7,8
0,node_id,query_id,subq_name,state,token_tag,state_value,state_alias,state_parentname,state_realname
1,1,1,MAIN,FROM,value,SUBQ.MAIN_FROM1 AS Ranking,,,
2,2,1,MAIN,FROM,token,"alias:Ranking, Database:SUBQ, Table:MAIN_FROM1",Ranking,SUBQ,MAIN_FROM1
3,3,1,MAIN,WHERE,value,WHERE Rank % 10 = 0;,,,
4,4,1,MAIN,SELECT,value,*,,,
5,5,1,MAIN,TABLE,token,NOCREATETB.query1,,NOCREATETB,query1
6,6,1,MAIN_FROM1,FROM,value,Table1,,,
7,7,1,MAIN_FROM1,FROM,token,"alias:, Database:, Table:Table1",,,Table1
8,8,1,MAIN_FROM1,SELECT,value,"*, (Dense_Rank() OVER (ORDER BY Column ASC)) A...",,,
9,9,1,MAIN_FROM1,TABLE,token,SUBQ.MAIN_FROM1,,SUBQ,MAIN_FROM1


In [7]:
pd.DataFrame(sas_query.from_to_property)

,0,1
0,from,to
1,2,1
2,1,3
3,3,4
4,4,5
5,7,6
6,6,8
7,8,9


In [34]:
"TABLE" in list(sas_query.sql_parsed_list['1'].structured_dict['MAIN'].keys())

False

In [37]:
sas_query.sql_parsed_list['1'].structured_dict['MAIN']['TABLE'] = {'token' : [['', 'QUERYID', 'query' + '1']], 'value': 'QUERYID.query' + '1'}

In [3]:
sas_query.sql_parsed_list['1'].structured_dict['MAIN']

{'FROM': {'token': [['Ranking', 'SUBQ', 'MAIN_FROM1']],
  'value': 'SUBQ.MAIN_FROM1 AS Ranking'},
 'SELECT': {'value': '*'},
 'TABLE': {'token': [['', 'QUERYID', 'query1']], 'value': 'QUERYID.query1'},
 'WHERE': {'value': 'WHERE Rank % 10 = 0;'}}

In [4]:
sas_query.sql_parsed_list['1'].structured_dict['MAIN_FROM1']

{'FROM': {'token': [[None, None, 'Table1']], 'value': 'Table1'},
 'SELECT': {'value': '*, (Dense_Rank() OVER (ORDER BY Column ASC)) AS Rank, b'},
 'TABLE': {'token': [['', 'SUBQ', 'MAIN_FROM1']], 'value': 'SUBQ.MAIN_FROM1'}}

In [20]:
from selenium import webdriver
driver = webdriver.Chrome() #開啟firefox

driver.get("https://mermaid-js.github.io/mermaid-live-editor/") #前往這個網址

# 尋找網頁中的搜尋框
inputElement = driver.find_element_by_xpath("//div[@class='view-lines']")
print(inputElement.text)
# 在搜尋框中輸入文字
inputElement.send_keys("Selenium")


# driver.close()#關閉瀏覽器

graph TD
  A[Christmas] -->|Get money| B(Go shopping)
  B --> C{Let me think}
  C -->|One| D[Laptop]
  C -->|Two| E[iPhone]
  C -->|Three| F[fa:fa-car Car]
    


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=83.0.4103.106)


In [21]:
re.sub("\\/.*?\\/", "", query4)

"\n\nPROC SQL;\n   CREATE TABLE Meow.HOSPITAL_5Y_01 AS \n   SELECT DISTINCT t1.CONTRACT_NO, \n         t1.CREATE_DATE,\n          t1.INSD_ID_SAS,\n          t2.APLY_NO,\n\t\t  t2.POLICY_NO AS POL_2, \n\t\t  t2.PROD_ID,\n\t      t2.OCR_DATE,\n\t\t  t2.E_STR_DATE,\n\t\t  t2.E_END_DATE,\n\t\t  t2.CLAM_AMT_CODE,\n\t\t  t2.CLAM_AMT_NAME,\n\t\t  t2.CLAM_CAT,\n\t\t  t2.OCR_RESN, \n          t2.BONE_CODE, \n          t2.BONE_NAME, \n\t\t  t2.OP_CODE,\n          t2.OP_NAME,\n\t\t  t2.PAY_DAY,\n\t\t  t2.PAY_AMT,\n\t\t  t2.PAY_STS\n      FROM Meow.POL_ALL_X_01 t1\n           LEFT JOIN QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS)\n      WHERE t1.CREATE_DATE > t2.ISSUE_DATE ;\nQUIT;\n\nPROC SQL;\n   CREATE TABLE WORK.HOSPITAL_5Y_02 AS \n   SELECT DISTINCT  t1.CONTRACT_NO, \n           t1.CREATE_DATE, \n          t1.INSD_ID_SAS,\n          t1.APLY_NO,\n\t\t  t1.POL_2 , \n\t\t  t1.PROD_ID,\n\t      t1.OCR_DATE,\n\t\t  t1.E_STR_DATE,\n\t\t  t1.E_END_DATE,\n\t\t  t1.OCR_RESN, \n          t1.

In [16]:
query4

"\n\nPROC SQL;\n   CREATE TABLE Meow.HOSPITAL_5Y_01 AS \n   SELECT DISTINCT t1.CONTRACT_NO, \n         t1.CREATE_DATE,\n          t1.INSD_ID_SAS,\n          t2.APLY_NO,\n\t\t  t2.POLICY_NO AS POL_2, \n\t\t  t2.PROD_ID,\n\t      t2.OCR_DATE,\n\t\t  t2.E_STR_DATE,\n\t\t  t2.E_END_DATE,\n\t\t  t2.CLAM_AMT_CODE,\n\t\t  t2.CLAM_AMT_NAME,\n\t\t  t2.CLAM_CAT,\n\t\t  t2.OCR_RESN, \n          t2.BONE_CODE, \n          t2.BONE_NAME, \n\t\t  t2.OP_CODE,\n          t2.OP_NAME,\n\t\t  t2.PAY_DAY,\n\t\t  t2.PAY_AMT,\n\t\t  t2.PAY_STS\n      FROM Meow.POL_ALL_X_01 t1\n           LEFT JOIN QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS)\n      WHERE t1.CREATE_DATE > t2.ISSUE_DATE ;\nQUIT;\n\nPROC SQL;\n   CREATE TABLE WORK.HOSPITAL_5Y_02 AS \n   SELECT DISTINCT  t1.CONTRACT_NO, \n           t1.CREATE_DATE, \n          t1.INSD_ID_SAS,\n          t1.APLY_NO,\n\t\t  t1.POL_2 , \n\t\t  t1.PROD_ID,\n\t      t1.OCR_DATE,\n\t\t  t1.E_STR_DATE,\n\t\t  t1.E_END_DATE,\n\t\t  t1.OCR_RESN, \n          t1.

In [3]:
sas_query.from_to_query

[['from', 'to'], ['6', '8']]

In [4]:
sas_query.from_to_property

[['from', 'to'],
 ['2', '1'],
 ['3', '1'],
 ['1', '4'],
 ['4', '5'],
 ['5', '6'],
 ['8', '7'],
 ['9', '7'],
 ['7', '10'],
 ['10', '11']]

In [5]:
for item in sas_query.node_dict.items():
    print(str(item[0]) + " : " + "Node_prop : " + str(item[1][3]) + "," + str(item[1][4]) + " \n    Node_value : " + str(item[1][5]))

1 : Node_prop : FROM,value 
    Node_value : Meow.POL_ALL_X_01 INNER JOIN QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS)
2 : Node_prop : FROM,token 
    Node_value : alias:, Database:Meow, Table:POL_ALL_X_01
3 : Node_prop : FROM,token 
    Node_value : alias:t2, Database:QUERY, Table:DTAAB001_Q
4 : Node_prop : WHERE,value 
    Node_value : WHERE t1.CREATE_DATE > t2.OCR_DATE ;
5 : Node_prop : SELECT,value 
    Node_value : DISTINCT t1.CONTRACT_NO,t1.INSD_ID_SAS,t2.APLY_NO
6 : Node_prop : TABLE,token 
    Node_value : MEOW.PAST_CLAIM_APLY_NO
7 : Node_prop : FROM,value 
    Node_value : MEOW.PAST_CLAIM_APLY_NO t1 INNER JOIN QUERY.DTAAA020_AE t2 ON (t1.APLY_NO = t2.APLY_NO) ;
8 : Node_prop : FROM,token 
    Node_value : alias:t1, Database:MEOW, Table:PAST_CLAIM_APLY_NO
9 : Node_prop : FROM,token 
    Node_value : alias:t2, Database:QUERY, Table:DTAAA020_AE
10 : Node_prop : SELECT,value 
    Node_value : DISTINCT t1.CONTRACT_NO,t1.INSD_ID_SAS,t2.APLY_NO, t2.DIAG_SER_NO, t2.CFM_ICD_

In [2]:
import markdown
text = """
# Title

Some text.

~~~mermaid
graph TB
A --> B
B --> C
~~~

Some other text.

~~~mermaid
graph TB
D --> E
E --> F
~~~
"""

html = markdown.markdown(text, extensions=['md_mermaid'])

print(html)

<h1>Title</h1>
<p>Some text.</p>
<div class="mermaid">
graph TB
A --> B
B --> C
</div>

<p>Some other text.</p>
<div class="mermaid">
graph TB
D --> E
E --> F
</div>

<script>mermaid.initialize({startOnLoad:true});</script>


In [237]:
class Mermaid_creator():
    
    def __init__(self):
        self.mermaid_txt = "graph TD;"
    
    # 00 Close Sentence
    def close_sentence(self):
        self.mermaid_txt += ";"
    
    # 01 Add Node    
    def add_node(self, shape = "", txt = "", node_id = ""):
        # Setting node id 
        self.mermaid_txt += str(node_id)
        
        # Setting texts
        if shape == "rectangular":
            self.mermaid_txt += "[" + txt +"]"
            
        elif shape == "roundedges":
            self.mermaid_txt += "(" + txt +")"
            
        elif shape == "circle":
            self.mermaid_txt += "((" + txt +"))"
            
        elif shape == "tag":
            self.mermaid_txt += ">" + txt +"]"
            
        elif shape == "diamond":
            self.mermaid_txt += "{" + txt +"}" 
            
            
    # 02 Add Link        
    def add_link(self, shape = "", txt = ""):
        
        # Setting relation texts
        if shape == "":
            print("please set a shape for line")
        
        # 01 Arrowhead
        elif shape == "arrowhead" :
            if txt == "" : 
                self.mermaid_txt += "-->"
            else:
                self.mermaid_txt += "-->" + "|" + txt + "|"
        
        # 02 Openlink
        elif shape == "openlink" :
            if txt == "" : 
                self.mermaid_txt += "---"
            else:
                self.mermaid_txt += "---" + "|" + txt + "|"
                
        # 03 Dotted link
        elif shape == "dottedlink" :
            if txt == "" : 
                self.mermaid_txt += "-.->"
            else:
                self.mermaid_txt += "-." + txt + ".->"
        
        # 03 Thick link
        elif shape == "thicklink" :
            if txt == "" : 
                self.mermaid_txt += "==>"
            else:
                self.mermaid_txt += "==" + txt + "==>"
                
    # 03 Subgraphs        
    def subgraph(self, state = "", txt = ""):
        # Setting relation texts
        if state == "" :
            print("please set a shape and text for subgraph")
            
        if state == "start":
            self.mermaid_txt += "subgraph " + txt + ";"
            
        if state == "end" and txt == "":
            self.mermaid_txt += "end; "
            
    # 04 Setting Style 
    def add_node_color(self, node_id, color) :
        # Setting node Color
        self.mermaid_txt += "\n style " + node_id + " fill:" + color + ";"

In [7]:
mc = Mermaid_creator()
mc.subgraph(state = "start", txt = "123456789")

In [8]:
mc = Mermaid_creator()

mc.subgraph(state = "start", txt = "123456789")
mc.add_node(shape = "rectangular", txt = "First Node", node_id = "1")
mc.add_link(shape = "arrowhead", txt = "")
mc.add_node(shape = "rectangular", txt = "Second Node", node_id = "2")
mc.close_sentence()

mc.add_node(shape = "roundedges", txt = "Third Node", node_id = "3")
mc.add_link(shape = "arrowhead", txt = "Some relations")
mc.add_node(shape = "rectangular", txt = "Second Node", node_id = "2")
mc.close_sentence()
mc.subgraph(state = "end")

mc.add_node(shape = "rectangular", txt = "Second Node", node_id = "2")
mc.add_link(shape = "dottedlink", txt = "table connect")
mc.add_node(shape = "diamond", txt = "Forth Node", node_id = "4")
mc.close_sentence()

mc.add_node(node_id = "4")
mc.add_link(shape = "dottedlink", txt = "table connect")
mc.add_node(node_id = "5")
mc.close_sentence()

In [9]:
mc.mermaid_txt

'graph TD;subgraph 123456789;1[First Node]-->2[Second Node];3(Third Node)-->|Some relations|2[Second Node];end; 2[Second Node]-.table connect.->4{Forth Node};4-.table connect.->5;'

In [10]:
sas_query.from_to_property
sas_query.from_to_query
sas_query.node_property

[['node_id',
  'query_id',
  'subq_name',
  'state',
  'token_tag',
  'state_value',
  'state_alias',
  'state_parentname',
  'state_realname'],
 ['1',
  '1',
  'MAIN',
  'FROM',
  'value',
  'Meow.POL_ALL_X_01 INNER JOIN QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS)',
  '',
  '',
  ''],
 ['2',
  '1',
  'MAIN',
  'FROM',
  'token',
  'alias:, Database:Meow, Table:POL_ALL_X_01',
  '',
  'Meow',
  'POL_ALL_X_01'],
 ['3',
  '1',
  'MAIN',
  'FROM',
  'token',
  'alias:t2, Database:QUERY, Table:DTAAB001_Q',
  't2',
  'QUERY',
  'DTAAB001_Q'],
 ['4',
  '1',
  'MAIN',
  'WHERE',
  'value',
  'WHERE t1.CREATE_DATE > t2.OCR_DATE ;',
  '',
  '',
  ''],
 ['5',
  '1',
  'MAIN',
  'SELECT',
  'value',
  'DISTINCT t1.CONTRACT_NO,t1.INSD_ID_SAS,t2.APLY_NO',
  '',
  '',
  ''],
 ['6',
  '1',
  'MAIN',
  'TABLE',
  'token',
  'MEOW.PAST_CLAIM_APLY_NO',
  '',
  'MEOW',
  'PAST_CLAIM_APLY_NO'],
 ['7',
  '2',
  'MAIN',
  'FROM',
  'value',
  'MEOW.PAST_CLAIM_APLY_NO t1 INNER JOIN QUERY.DTAAA020_

In [43]:
import pandas as pd
node_property

,node_id,query_id,subq_name,state,token_tag,state_value,state_alias,state_parentname,state_realname
0,1,1,MAIN,FROM,value,Meow.POL_ALL_X_01 INNER JOIN QUERY.DTAAB001_Q ...,,,
1,2,1,MAIN,FROM,token,"alias:, Database:Meow, Table:POL_ALL_X_01",,Meow,POL_ALL_X_01
2,3,1,MAIN,FROM,token,"alias:t2, Database:QUERY, Table:DTAAB001_Q",t2,QUERY,DTAAB001_Q
3,4,1,MAIN,WHERE,value,WHERE t1.CREATE_DATE > t2.OCR_DATE ;,,,
4,5,1,MAIN,SELECT,value,"DISTINCT t1.CONTRACT_NO,t1.INSD_ID_SAS,t2.APLY_NO",,,
5,6,1,MAIN,TABLE,token,MEOW.PAST_CLAIM_APLY_NO,,MEOW,PAST_CLAIM_APLY_NO
6,7,2,MAIN,FROM,value,MEOW.PAST_CLAIM_APLY_NO t1 INNER JOIN QUERY.DT...,,,
7,8,2,MAIN,FROM,token,"alias:t1, Database:MEOW, Table:PAST_CLAIM_APLY_NO",t1,MEOW,PAST_CLAIM_APLY_NO
8,9,2,MAIN,FROM,token,"alias:t2, Database:QUERY, Table:DTAAA020_AE",t2,QUERY,DTAAA020_AE
9,10,2,MAIN,SELECT,value,"DISTINCT t1.CONTRACT_NO,t1.INSD_ID_SAS,t2.APLY...",,,


In [211]:
node_property = pd.DataFrame(sas_query.node_property[1:], columns = sas_query.node_property[0])
node_property[(node_property["node_id"] == "3") & (node_property["token_tag"] == "token")]

,node_id,query_id,subq_name,state,token_tag,state_value,state_alias,state_parentname,state_realname
2,3,1,MAIN,FROM,token,"alias:t2, Database:QUERY, Table:DTAAB001_Q",t2,QUERY,DTAAB001_Q


In [90]:
node_property[node_property["node_id"] == "3"]["node_id"].tolist()

['3']

In [238]:
import pandas as pd
class Sqltoflowchart():
    
    def __init__(self, from_to_node, from_to_query, node_property):
        self.from_to_node = from_to_node
        self.from_to_query = from_to_query
        self.node_property = pd.DataFrame(node_property[1:], columns= node_property[0])
        self.mc = Mermaid_creator()
    
    # 00_1 regrex Setting (upper and lower no different)
    def upper_lower_regrex(self, string):
        regex_txt = ""
        for c in string:
            if c == " " : 
                regex_txt += " "
            else:
                regex_txt += "[" + c.lower() + c.upper() + "]"
        return(regex_txt)
    
    # 01 Create node txt
    def Create_nodetxt(self, node_id, node):
        token_tag = node["token_tag"].tolist()[0]
        node_value = ""
        node_value += "<b>" + node["state"].tolist()[0] + "</b>"
        
        if token_tag == "token" :
            node_value += "<br>" + node["state_parentname"].tolist()[0] + "." + node["state_realname"].tolist()[0] + ", " + node["state_alias"].tolist()[0]
        
        ## Add <center> </center>
        node_value = "<center>" + node_value + "</center>"
        
        return node_value
    
    # 02 Create relation txt
    def Create_relationtxt(self, node_id, node):
        token_tag = node["token_tag"].tolist()[0]
        relation_value = ""
        
        if token_tag == "token" : 
            print("Empty relation (Class Sqltoflowchart - def Create_relationtxt)")
        else :
            relation_value = node[ "state_value" ].tolist()[0]
            relation_value = re.sub("\\(|\\)", " ", relation_value)
            relation_value = re.sub(";", "", relation_value)
            relation_value = re.sub(self.upper_lower_regrex("INNER JOIN "), "<b>INNER JOIN </b>", relation_value)
            relation_value = re.sub(self.upper_lower_regrex("LEFT JOIN "), "<b>LEFT JOIN </b>", relation_value)
            relation_value = re.sub(self.upper_lower_regrex("RIGHT JOIN "), "<b>RIGHT JOIN </b>", relation_value)
            relation_value = re.sub(self.upper_lower_regrex("OUTER JOIN "), "<b>OUTER JOIN </b>", relation_value)
            relation_value = re.sub(self.upper_lower_regrex("CROSS JOIN "), "<b>CROSS JOIN </b>", relation_value)
            relation_value = re.sub(self.upper_lower_regrex("DISTINCT "), "<b>DISTINCT </b>", relation_value)
            relation_value = re.sub(self.upper_lower_regrex("ON "), "ON <br>", relation_value)
            relation_value = re.sub(self.upper_lower_regrex(","), ", <br>", relation_value)
#             print("Need txt relation (Class Sqltoflowchart - def Create_relationtxt)")
        ## Add <center> </center>
        relation_value = "<center>" + relation_value + "</center>"
        
        return(relation_value)    
    
    #03 Create mermaid plot
    def mermaid_plot(self):
        #### 03_2 Initial Subgraph start drawing
        subgraph_sign = 0 # Record the subgraph , start->set 1, end->set 0
        
        
        #01 Node relation create
        for from_to in self.from_to_node :
            from_id = ""
            to_id = ""
            from_node = ""
            to_node = ""
            node_txt = ""
            relation = ""
            
            if from_to[0] == "from" :
                print("columnnames" + str(from_to))
                continue
            
            
            # 03_1 Initial setting from and to node setting 
            from_id = from_to[0]
            to_id = from_to[1]
            from_node = self.node_property[self.node_property["node_id"] == from_id]
            to_node = self.node_property[self.node_property["node_id"] == to_id]
            
            #03_2 Subgraph start drawing(from_node) ===============================
            
            if from_node["state"].tolist()[0] == "FROM" and from_node["token_tag"].tolist()[0] == "token" and subgraph_sign == 0 :
                subgraph_sign = 1
                self.mc.subgraph(state = "start", txt = "<needtobereplacetable>")
                
                
            #03_3 From node create (from_node)===============================
            node_txt = self.Create_nodetxt(node_id = from_id, node = from_node)
            if from_node["token_tag"].tolist()[0] == "token" :
                self.mc.add_node(shape = "roundedges", txt = node_txt, node_id = from_id)
            else : 
                self.mc.add_node(shape = "diamond", txt = node_txt, node_id = from_id)
            
            
            #03_4 From relation create (from_node)===============================
            relation_txt = self.Create_relationtxt(node_id = from_id, node = from_node)
            if from_node["token_tag"].tolist()[0] == "token" :
                self.mc.add_link(shape = "arrowhead", txt = "")
            else : 
                self.mc.add_link(shape = "arrowhead", txt = relation_txt)
            
            
            
            #03_5 To node create (to_node)===============================
            node_txt = self.Create_nodetxt(node_id = to_id, node = to_node)
            if to_node["token_tag"].tolist()[0] == "token" :
                self.mc.add_node(shape = "roundedges", txt = node_txt, node_id = to_id)
            else : 
                self.mc.add_node(shape = "diamond", txt = node_txt, node_id = to_id)
            
            #03_6 End Line ===============================
            self.mc.close_sentence()
            
            
            #03_7 Subgraph end drawing (to_node) ===============================
            ## 01 "TABLE" state, "token" token_tag setting end line
            ## 02 Replace the  mermaid_txt <needtobereplacetable> replace to TABLE name 
            if to_node["state"].tolist()[0] == "TABLE" and to_node["token_tag"].tolist()[0] == "token" and subgraph_sign == 1 :
                subgraph_sign = 0
                self.mc.subgraph(state = "end")
                self.mc.mermaid_txt = re.sub("\\<needtobereplacetable\\>", to_node["state_value"].tolist()[0], self.mc.mermaid_txt)
                print("replace relation (Class Sqltoflowchart - def Create_relationtxt)")
            
        #02 query relation create
        for from_to_query in self.from_to_query :
            from_id = ""
            to_id = ""
            if from_to_query[0] == "from" :
                print("columnnames" + str(from_to_query))
                continue
            print("columnnames" + str(from_to_query))
            # 04_1 Initial setting from and to node setting 
            from_id = from_to_query[0]
            to_id = from_to_query[1]
            
            self.mc.add_node(node_id = from_id)
            self.mc.add_link(shape = "dottedlink", txt = "")
            self.mc.add_node(node_id = to_id)
            self.mc.close_sentence()
            
    #04 Draw mermaid plot
    def mermaid_drawnode(self, keyword, token_tag, color):
        draw_nodes = self.node_property[(self.node_property["state_parentname"] == keyword) & (self.node_property["token_tag"] == token_tag)]
        for node_id in draw_nodes["node_id"]:
            self.mc.add_node_color(node_id, color)

In [239]:
sfc = Sqltoflowchart(sas_query.from_to_property,
sas_query.from_to_query,
sas_query.node_property)

In [214]:
sas_query.from_to_query

[['from', 'to'], ['6', '8']]

In [240]:
sfc.mermaid_plot()
sfc.mermaid_drawnode(keyword = "QUERY", token_tag = "token", color = "#28FF28")

columnnames['from', 'to']
Empty relation (Class Sqltoflowchart - def Create_relationtxt)
Empty relation (Class Sqltoflowchart - def Create_relationtxt)
replace relation (Class Sqltoflowchart - def Create_relationtxt)
Empty relation (Class Sqltoflowchart - def Create_relationtxt)
Empty relation (Class Sqltoflowchart - def Create_relationtxt)
replace relation (Class Sqltoflowchart - def Create_relationtxt)
columnnames['from', 'to']
columnnames['6', '8']


In [241]:
sfc.mc.mermaid_txt

'graph TD;subgraph MEOW.PAST_CLAIM_APLY_NO;2(<center><b>FROM</b><br>Meow.POL_ALL_X_01, </center>)-->1{<center><b>FROM</b></center>};3(<center><b>FROM</b><br>QUERY.DTAAB001_Q, t2</center>)-->1{<center><b>FROM</b></center>};1{<center><b>FROM</b></center>}-->|<center>Meow.POL_ALL_X_01 <b>INNER JOIN </b>QUERY.DTAAB001_Q t2 ON <br> t1.INSD_ID_SAS = t2.OCR_ID_SAS </center>|4{<center><b>WHERE</b></center>};4{<center><b>WHERE</b></center>}-->|<center>WHERE t1.CREATE_DATE > t2.OCR_DATE </center>|5{<center><b>SELECT</b></center>};5{<center><b>SELECT</b></center>}-->|<center><b>DISTINCT </b>t1.CONTRACT_NO, <br>t1.INSD_ID_SAS, <br>t2.APLY_NO</center>|6(<center><b>TABLE</b><br>MEOW.PAST_CLAIM_APLY_NO, </center>);end; subgraph WORK.PAST_CLAIM_ICDS;8(<center><b>FROM</b><br>MEOW.PAST_CLAIM_APLY_NO, t1</center>)-->7{<center><b>FROM</b></center>};9(<center><b>FROM</b><br>QUERY.DTAAA020_AE, t2</center>)-->7{<center><b>FROM</b></center>};7{<center><b>FROM</b></center>}-->|<center>MEOW.PAST_CLAIM_APLY_NO t

In [79]:
sas_query.from_to_property

[['from', 'to'],
 ['2', '1'],
 ['3', '1'],
 ['1', '4'],
 ['4', '5'],
 ['5', '6'],
 ['8', '7'],
 ['9', '7'],
 ['7', '10'],
 ['10', '11']]

In [75]:
regex_txt = ""
for c in "INNER JOIN ":
    if c == " " : 
        regex_txt += " "
    else:
        regex_txt += "[" + c.lower() + c.upper() + "]"
print(regex_txt)

[iI][nN][nN][eE][rR] [jJ][oO][iI][nN] 


In [78]:
sfc.Create_relationtxt(node_id = "7", node = sfc.node_property[sfc.node_property["node_id"] == "7"])

'MEOW.PAST_CLAIM_APLY_NO t1 <b>INNER JOIN </b>QUERY.DTAAA020_AE t2 ON <br> t1.APLY_NO = t2.APLY_NO  ;'

## mermaid plot 
### supermarkdown

In [153]:
re.sub("\\<needtobereplacetable\\>", "xxxx", "<needtobereplacetable>;")

'xxxx;'

In [64]:
re.sub( regex_txt, "", " FROM Meow.POL_ALL_X_01 INNER JOIN QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS) ")

' FROM Meow.POL_ALL_X_01 QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS) '

In [93]:
from SuperMarkdown import SuperMarkdown

supermd = SuperMarkdown.SuperMarkdown()

In [10]:
import markdown

text = """

~~~mermaid
graph LR
A[Zip file]-->B[\"R (abc) functions\"]
A[Zip file]-->C[manual<br>for R<br>functions]
A[Zip file]-->D[vingettes]
D[vingettes]-->E[data]
D[vingettes]-->F[figures]
D[vingettes]-->G((R<br>markdown<br>file))
A[Zip file]-->H((README<br>file))

style B fill:red
~~~
"""

html = markdown.markdown(text, extensions=['md_mermaid'])

print(html)

<div class="mermaid">
graph LR
A[Zip file]-->B["R (abc) functions"]
A[Zip file]-->C[manual<br>for R<br>functions]
A[Zip file]-->D[vingettes]
D[vingettes]-->E[data]
D[vingettes]-->F[figures]
D[vingettes]-->G((R<br>markdown<br>file))
A[Zip file]-->H((README<br>file))

style B fill:red
</div>

<script>mermaid.initialize({startOnLoad:true});</script>


True